In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from datetime import datetime, date

In [2]:
def grab_symbols():
    # Grab S&P Symbols from Wikipedia or local HTML File
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks'
    tickers = pd.read_html('./tickers.html')[0]
    tickers = tickers.Symbol.to_list()
    tickers = [i.replace('.','-') for i in tickers]
    return tickers 

print('Grab symbols function defined...')

Grab symbols function defined...


In [3]:
def yahoo_prices(tickers, current_day):
    # Return a DataFrame with dates, symbols and prices
    if not isinstance(tickers,list):
        return None

    years_back = 5
    end_dt = datetime.strptime(current_day, '%Y-%m-%d').date()
    start_dt = date(end_dt.year - years_back, end_dt.month, end_dt.day)

    data = yf.download(tickers, start=start_dt, end=end_dt)
    data = data.loc[(slice(None)),(slice(None),slice(None))].copy()
    data = data.stack()
    data = data.reset_index()
    data.rename(columns={'level_1': 'Symbol'}, inplace=True)
    data.set_index('Date', inplace=True)
    return data

print('Yahoo_prices function defined...')    

Yahoo_prices function defined...


In [4]:
# Generate our indicators for Buy Signals for each of our assets
# df: contains one company (symbol) of data
def add_RSI_buy_signals(df):
    # Sanity check for our indicators to work
    if len(df) < 250:
        return None
    
    df['MA200'] = df['Adj Close'].rolling(window=200).mean()
    df['Price_change'] = df['Adj Close'].pct_change()
    df['Upmove']   = df['Price_change'].apply(lambda x: x if x > 0 else 0)
    df['Downmove'] = df['Price_change'].apply(lambda x: abs(x) if x < 0 else 0)
    df['Avg_up']   = df['Upmove'].ewm(span=19).mean()
    df['Avg_down']   = df['Downmove'].ewm(span=19).mean()
    df = df.dropna().copy()
    df['RS'] = df['Avg_up'] / df['Avg_down']
    df['RSI'] = df['RS'].apply(lambda x: 100 - (100 / (x + 1)))
    df.loc[(df['Adj Close'] > df['MA200']) & (df['RSI'] < 30), 'Buy_signal' ] = 'Yes'
    df.loc[(df['Adj Close'] <= df['MA200']) | (df['RSI'] >= 30), 'Buy_signal' ] = 'No'

    return df
    
print('add_RSI_buy_signals function defined....')

add_RSI_buy_signals function defined....


In [90]:
def order_simuator(df):
    max_hold_days = 10    # hold position for a maximum of 10 days
    stop_loss_percentage = -0.03
    target_percentage = 0.1
    # Order actions : Enter, Skip, Close, Stop, Target (a buy signal will be skipped if already in position)
    df['Order'] = ''

    in_position_start_index = None
    for index, row in df.iterrows():
        # Check for Enter position conditions
        if row.Buy_signal == 'Yes' and in_position_start_index is None:
            df.at[index, 'Order'] = 'Enter'
            in_position_start_index = index
        elif row.Buy_signal == 'Yes' and in_position_start_index is not None:
            df.at[index, 'Order'] = 'Skip'
        
        # Check for exit position conditions
        if in_position_start_index is not None:
            if (index - in_position_start_index).days >= max_hold_days:
                df.at[index, 'Order'] = 'Close'
                in_position_start_index = None
            elif index > in_position_start_index: 
                buy_price = df.at[in_position_start_index, 'Adj Close']
                current_price = row['Adj Close']
                percent_change = (current_price - buy_price) / buy_price
                if percent_change < stop_loss_percentage:
                    df.at[index, 'Order'] = 'Stop'
                    in_position_start_index = None
                if percent_change > target_percentage:
                    df.at[index, 'Order'] = 'Target'
                    in_position_start_index = None
        #else:
        #    df.at[index, 'Position'] = '-'

        #     if index >= in_position_start_index:
        #         df.at[index, 'Position'] = 'Close'
        #         in_position_start_index = None
    return df

print('Trade simulator function defined....')

Trade simulator function defined....


In [ ]:
def execution_simulator():
    # Using portfolio cash, percentage to buy position, etc....
    pass

In [26]:
# Grab symbols and get prices from yahoo
tickers = grab_symbols()
test_length = 5
all_assets = yahoo_prices(tickers[0:test_length], '2023-01-31')

[*********************100%***********************]  5 of 5 completed


In [91]:
# Grab prices, add indicators and signals and store all dataframes in a list
multi_assets = []
len_tickers = len(tickers[0:test_length])
for i in range(len_tickers):
    frame = all_assets.loc[all_assets.Symbol == tickers[i]].copy()
    frame = add_RSI_buy_signals(frame)
    multi_assets.append(frame)

# Run trade simlator
test_ticker = 2
multi_assets[test_ticker] = order_simuator(multi_assets[test_ticker])

frame = multi_assets[test_ticker] 
frame.loc[(frame.Order != '') | (frame.Buy_signal == 'Yes'), ['Symbol', 'Adj Close', 'Volume', 'Buy_signal', 'Order']].tail(50)

,Symbol,Adj Close,Volume,Buy_signal,Order
Date,,,,,
2018-12-24 00:00:00-05:00,ABT,61.214024,4026200,Yes,Enter
2018-12-31 00:00:00-05:00,ABT,67.535240,6094300,No,Target
2019-04-17 00:00:00-04:00,ABT,68.644112,11212400,Yes,Enter
2019-04-18 00:00:00-04:00,ABT,69.623657,9712200,Yes,Skip
2019-04-29 00:00:00-04:00,ABT,73.720818,4355000,No,Close
2020-02-24 00:00:00-05:00,ABT,80.671623,5667200,Yes,Enter
2020-02-25 00:00:00-05:00,ABT,77.335342,7021800,No,Stop
2021-09-27 00:00:00-04:00,ABT,117.601425,6481200,Yes,Enter
2021-09-28 00:00:00-04:00,ABT,115.806885,5855200,Yes,Skip


In [86]:
buy_p = 68.644112
cur_p = 73.720818
(cur_p - buy_p) / buy_p

0.0739569039803441

In [81]:
specific_date = pd.to_datetime('2022-01-31')
frame.loc[:specific_date, ['Symbol', 'Adj Close', 'Open', 'Buy_signal', 'Order']].tail(25)

C:\Users\boivi\AppData\Local\Temp\ipykernel_15928\132631299.py:2: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  frame.loc[:specific_date, ['Symbol', 'Adj Close', 'Open', 'Buy_signal', 'Order']].tail(25)


,Symbol,Adj Close,Open,Buy_signal,Order
Date,,,,,
2021-12-27 00:00:00-05:00,ABT,138.498352,139.820007,No,
2021-12-28 00:00:00-05:00,ABT,137.529068,141.580002,No,
2021-12-29 00:00:00-05:00,ABT,138.234009,140.470001,No,
2021-12-30 00:00:00-05:00,ABT,138.047974,141.410004,No,
2021-12-31 00:00:00-05:00,ABT,137.793427,141.000000,No,
2022-01-03 00:00:00-05:00,ABT,136.128998,139.429993,No,
2022-01-04 00:00:00-05:00,ABT,132.927490,139.500000,No,
2022-01-05 00:00:00-05:00,ABT,132.330246,135.149994,No,
2022-01-06 00:00:00-05:00,ABT,132.310654,135.000000,No,


In [82]:
buy_p = 121.655258
cur_p = 121.114861
(cur_p - buy_p) / buy_p

-0.004442035707161943

In [69]:
specific_date = pd.to_datetime('2022-01-31')
frame.loc[specific_date, ['Symbol', 'Adj Close', 'Volume', 'Buy_signal', 'Position']].tail(30)

C:\Users\boivi\AppData\Local\Temp\ipykernel_15928\3773098246.py:2: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  frame.loc[specific_date, ['Symbol', 'Adj Close', 'Volume', 'Buy_signal', 'Position']].tail(30)


KeyError: "['Position'] not in index"

In [ ]:
# print(f'Multi assets list: {len(multi_assets)}')
# print(f'Testing with {test_length} out of {len(tickers)} total tickers')

# test_ticker = 1
# print(f'Symbol {test_ticker}: {tickers[test_ticker]}')
# frame = multi_assets[test_ticker] 
# frame.loc[frame.Buy_signal == 'Yes'].head()
